In [ ]:
#Setting up the code, importing all necessary packages and data

import pandas as pd
import numpy as np
from Bio import Entrez
import os
import time
Entrez.email = "spatuzzi@stud.uni-heidelberg.de"

#Modify this string based on your working directory in which your file is stored

WorkingDirectory = '/Users/darthvader/Desktop/Heidelberg/1_MoBi_Master/Praktikum_Bromham/Data'
os.chdir(WorkingDirectory)
Phy_genes = pd.read_csv(WorkingDirectory+'/Phy_genes.csv')

In [ ]:
#Data manipulation, establish global objects that will be needed later

#Dictionary to pair each gene with its "product" name
Product_dict = {"12S":["12S ribosomal RNA"], "16S":["16S ribosomal RNA"],"BDNF":["brain-derived neurotrophic factor"], "CXCR4":["chemokine receptor 4"],
            "cytb":["cytochrome b"], "H3A":["histone+H3", "histone H3a"], "NCX1":["sodium/calcium exchanger 1"], "ND1":["NADH dehydrogenase subunit 1",
            "ND1"], "ND2":["NADH dehydrogenase subunit 2"], "POMC":["proopiomelanocortin"], "RAG1":["recombination activating protein 1"], "RHOD":["rhodopsin"],
             "SIA":["seventh in absentia"], "SLC8A3":["solute carrier family 8 member 3"], "TYR":["tyrosinase"]}
Product_dict_key = Product_dict.keys()

#Edit the Phy_genes Dataframe
#We change to row names to the scientific names in order to use the .loc method, but do not get rid of the "Scientific name" 
#column because we will revert to a numbered index when we have multiple sequences for some gene-organism match
Phy_genes.columns = ['Scientific name', '12S', '16S', 'BDNF', 'CXCR4', 'cytb', 'H3A', 'NCX1',
       'ND1', 'ND2', 'POMC', 'RAG1', 'RHOD', 'SIA', 'SLC8A3', 'TYR']



In [ ]:
#Replace empty values with "None"

Phy_genes.index = Phy_genes.iloc[:,0].values

for name in Phy_genes.index:
    for gene in Phy_genes.columns:
        if(Phy_genes.loc[name, gene].count("-") == len(Phy_genes.loc[name, gene])):
                Phy_genes.loc[name,gene] = None


In [ ]:
#Create new Dataframes to fill out with new results
#Phy_genes_update: A copy of Phy_genes with the addition of new data
#new_seq_df: Datframe containing ONLY the new data

Phy_genes_update = Phy_genes.copy()
new_seq_df = pd.DataFrame( np.empty((len(Phy_genes.index), len(Phy_genes.columns)),dtype=pd.Timestamp), index = Phy_genes.index, columns = Phy_genes.columns)


In [ ]:
# Code for iteration

#Iterate over ever species
for name in Phy_genes.index:
    
    #Iterate over every Gene
    for gene in Product_dict_key:
        
        #Check if there is already a sequence in this position by removing all "gaps"
        #If it's only gaps, continue
        
        if(Phy_genes.loc[name, gene] == None):
            
            #To keep better track in the console, announce every time gene and organism
            print("Name:" + name + ", Gene:" + gene )

            #Concatenate the term for the Entrez function and preoare empty list for the results
            
            term = name + "[Orgn] AND " + Product_dict[gene][0] + "[Prd]"
            ID_list = []
            
            #This iteration can easily be interrupted by a temporary connection issue at this step
            #The following chunk of code ensures that the code is run multiple times (up to 100 times) to attempt to download entrez IDs 

            max_tries = 100
            for i in range(max_tries):
                try:
                    time.sleep(0.1) 
                    
                    #Find IDs that match the "product" string
                    
                    handle = Entrez.esearch(db="nucleotide", term = term, idtype="acc")
                    record = Entrez.read(handle)
                    ID_list = record["IdList"]
                    print(ID_list)

                    break
                except Exception:
                    print("retry")
                    continue

            if(len(ID_list) > 0):

                #Take sequence for each ID with Entrez.efetch function
                #Edit the downloaded string object to only contain the coding sequence

                seq_list = []
                Seq_clean = ""
                for ID in ID_list: 

                    handle = Entrez.efetch(db="nucleotide", id=ID, rettype="gb", retmode="text")
                    Gene_entry = handle.read()
                    Seq_raw = Gene_entry[Gene_entry.find("translation") + 13:Gene_entry.find("ORIGIN")-2]
                    Seq_raw = "".join(Seq_raw.split("\n"))
                    Seq_clean = Seq_raw
                    for remove in " 1234567890/":
                        Seq_clean = Seq_clean.replace(remove, "").upper()

                    seq_list.append(Seq_clean)
                    
                
                #"Empty" result lists still contain a warning string, in which case they must not be appended to the result
                if(len(seq_list) > 0 and seq_list != ['EDIDPARAMETERISEMPTY.']):
                    Phy_genes_update.loc[name, gene] = seq_list
                    new_seq_df.loc[name, gene] = seq_list
                    print(" Replaced:" + gene + " for " + name)
    

In [ ]:
# Write files for MAFFT
import os 

#Create new directory
os.system("mkdir New_MAlign_AA")
os.chdir(WorkingDirectory + "/New_MAlign_AA")

new_seq_df.loc[:,"Scientific name"] = new_seq_df.index

#Iterate over every gene, for each write a file with each new sequence for each organism. Organisms that have multiple sequences for the same gene 
# will have an  additional cypher (1,2,3...etc.) to differentiate them.
for gene in Phy_genes.columns[1:17]:
    
    with open('New_alignment_sequences_AA_'+gene, 'w') as f:
        seq_str = ""
        for name in Phy_genes.index:
            if(new_seq_df.loc[name, gene] != None):
                
                line_list = ["> " + name + " "+ str(i)+ "\n" + new_seq_df.loc[name, gene][i] for i in range(len(new_seq_df.loc[name, gene]))]
                
                line =("\n".join(line_list))
                f.write(line + "\n")
                              
    
    
    